## Deep Researcher
### Using Orchestrator And Synthesizer
- given a topic creates a plan
  - i.e. sections
- for every section tools are available
  - search arxiv
  - search internet (tavilysearch)
- summarizes information
- checks if revision is required or more searches are required
- creates final summary

In [15]:
import os
from dotenv import load_dotenv
import operator

from pydantic import BaseModel, Field
from typing import List, Optional
from typing_extensions import Annotated, Literal, TypedDict

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

from langgraph.types import interrupt, Command
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

from langgraph.checkpoint.memory import MemorySaver

from IPython.display import display, Markdown, Image

In [17]:
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o",
    max_tokens=500,
    openai_api_key=os.environ["OPENAI_API_KEY"],
)

deepseek_llm = ChatGroq(
    temperature=0,
    model="deepseek-r1-distill-qwen-32b",
    max_tokens=2048,
    groq_api_key=os.environ["GROQ_API_KEY"],
    max_retries=3,
)


# google_llm = ChatGoogleGenerativeAI(
#     temperature=0,
#     model="gemini-1.5-flash",
#     max_output_tokens=1024,
#     google_api_key=os.environ["GOOGLE_API_KEY"],
# )

In [18]:
class Section(BaseModel):
    title: str = Field(description="The title for this section of the report.")
    overview: str = Field(description="Brief overview of the main topics and concepts to be covered in this section.")
    
class Sections(BaseModel):
    sections: List[Section] = Field(description="Sections to be included in the report.")


thinker = deepseek_llm.with_structured_output(Sections)

In [19]:
o = planner.invoke(
        [
            SystemMessage(content="Generate a plan for the report."),
            HumanMessage(content=f"Here is the report topic: what is life"),
        ]
    )

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '\n\n</think>\n\n<response>\n{\n  "name": "Sections",\n  "arguments": {\n    "sections": [\n      {\n        "title": "Introduction to Life",\n        "overview": "An overview of the concept of life, its definition, and its significance in the universe."\n      },\n      {\n        "title": "Biological Perspective",\n        "overview": "Exploring life from a biological standpoint, including the characteristics of living organisms and the processes of life."\n      },\n      {\n        "title": "Philosophical and Spiritual Views",\n        "overview": "Examining how different philosophies and religions interpret the meaning and essence of life."\n      },\n      {\n        "title": "The Evolution of Life",\n        "overview": "A discussion on the origi

In [22]:
print('''\n\n</think>\n\n<response>\n{\n  "name": "Sections",\n  "arguments": {\n    "sections": [\n      {\n        "title": "Introduction to Life",\n        "overview": "An overview of the concept of life, its definition, and its significance in the universe."\n      },\n      {\n        "title": "Biological Perspective",\n        "overview": "Exploring life from a biological standpoint, including the characteristics of living organisms and the processes of life."\n      },\n      {\n        "title": "Philosophical and Spiritual Views",\n        "overview": "Examining how different philosophies and religions interpret the meaning and essence of life."\n      },\n      {\n        "title": "The Evolution of Life",\n        "overview": "A discussion on the origins of life, evolution, and the diversity of life forms on Earth."\n      },\n      {\n        "title": "Challenges and Future of Life",\n        "overview": "Addressing current challenges to life, such as environmental issues, and contemplating the future of life on Earth and beyond."\n      }\n    ]\n  }\n}\n</response>''')



</think>

<response>
{
  "name": "Sections",
  "arguments": {
    "sections": [
      {
        "title": "Introduction to Life",
        "overview": "An overview of the concept of life, its definition, and its significance in the universe."
      },
      {
        "title": "Biological Perspective",
        "overview": "Exploring life from a biological standpoint, including the characteristics of living organisms and the processes of life."
      },
      {
        "title": "Philosophical and Spiritual Views",
        "overview": "Examining how different philosophies and religions interpret the meaning and essence of life."
      },
      {
        "title": "The Evolution of Life",
        "overview": "A discussion on the origins of life, evolution, and the diversity of life forms on Earth."
      },
      {
        "title": "Challenges and Future of Life",
        "overview": "Addressing current challenges to life, such as environmental issues, and contemplating the future of life o

In [ ]:
class State(TypedDict):
    topic: str
    sections: List[str]